In [1]:
import pandas as pd
import datetime as dt

In [2]:
th_users = pd.read_csv(r"C:\Users\HP\OneDrive\Desktop\Data Science\Assignment\Assignment_3\takehome_users.csv",encoding='cp1252',index_col="object_id")
th_user_engagement = pd.read_csv(r"C:\Users\HP\OneDrive\Desktop\Data Science\Assignment\Assignment_3\takehome_user_engagement.csv",encoding='cp1252',index_col="user_id")

In [3]:
th_users.head(2)

,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
object_id,,,,,,,,,
1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0


In [4]:
th_users.shape

(12000, 9)

In [5]:
th_user_engagement

,time_stamp,visited
user_id,,
1,2014-04-22 03:53:30,1
2,2013-11-15 03:45:04,1
2,2013-11-29 03:45:04,1
2,2013-12-09 03:45:04,1
2,2013-12-25 03:45:04,1
...,...,...
11996,2013-09-06 06:14:15,1
11997,2013-01-15 18:28:37,1
11998,2014-04-27 12:45:16,1


In [6]:
th_user_engagement.isna().count()

time_stamp    207917
visited       207917
dtype: int64

In [7]:
visit_grp = th_user_engagement.groupby("user_id").visited.count()
visit_grp

user_id
1         1
2        14
3         1
4         1
5         1
         ..
11996     1
11997     1
11998     1
11999     1
12000     1
Name: visited, Length: 8823, dtype: int64

In [8]:
users = visit_grp[ visit_grp > 2 ]
th_user_engag_upd = (users.to_frame()).join(th_user_engagement,rsuffix='_y').drop("visited_y",axis=1)

In [9]:
print(th_user_engag_upd.index.unique())
print(len(th_user_engag_upd.index.unique()))

Int64Index([    2,    10,    20,    33,    42,    43,    50,    53,    59,
               60,
            ...
            11961, 11964, 11965, 11967, 11969, 11975, 11980, 11981, 11988,
            11991],
           dtype='int64', name='user_id', length=2248)
2248


In [10]:
def day_count(time_data):
    return dt.datetime.strptime(time_data, "%Y-%m-%d %H:%M:%S").date()

In [11]:
th_user_engag_upd["date"] = th_user_engag_upd.time_stamp.apply(day_count)
th_user_engag_upd

,visited,time_stamp,date
user_id,,,
2,14,2013-11-15 03:45:04,2013-11-15
2,14,2013-11-29 03:45:04,2013-11-29
2,14,2013-12-09 03:45:04,2013-12-09
2,14,2013-12-25 03:45:04,2013-12-25
2,14,2013-12-31 03:45:04,2013-12-31
...,...,...,...
11991,5,2014-04-20 14:22:45,2014-04-20
11991,5,2014-04-25 14:22:45,2014-04-25
11991,5,2014-04-28 14:22:45,2014-04-28


In [12]:
adopted_user_list=[]
def calc(user_id_grp):
    user_id_grp["date_shift"] = user_id_grp["date"].shift(1)
    x = (user_id_grp["date"]-user_id_grp["date_shift"]).fillna("0 days")
    y = [int(str(i).split()[0]) for i in x.to_list()]
    for i in range(1,len(y)-1):
        if y[i]+y[i+1] < 8:
            adopted_user_list.append(int(user_id_grp.index[0]))
            break

In [13]:
th_user_engag_upd.groupby("user_id").apply(calc)

""


In [14]:
len(adopted_user_list)

1656

In [15]:
th_users.head(2)

,creation_time,name,email,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id
object_id,,,,,,,,,
1,2014-04-22 03:53:30,Clausen August,AugustCClausen@yahoo.com,GUEST_INVITE,1.398139e+09,1,0,11,10803.0
2,2013-11-15 03:45:04,Poole Matthew,MatthewPoole@gustr.com,ORG_INVITE,1.396238e+09,0,0,1,316.0


In [16]:
for i in th_users.index:
    if i in adopted_user_list:
        th_users.loc[i,"adopted_user"]=1
    else:
        th_users.loc[i,"adopted_user"]=0

In [18]:
df = th_users.copy()
df.reset_index(inplace=True)
df.drop(["object_id","name","email"],axis=1,inplace= True)
df.head(2)

,creation_time,creation_source,last_session_creation_time,opted_in_to_mailing_list,enabled_for_marketing_drip,org_id,invited_by_user_id,adopted_user
0,2014-04-22 03:53:30,GUEST_INVITE,1.398139e+09,1,0,11,10803.0,0.0
1,2013-11-15 03:45:04,ORG_INVITE,1.396238e+09,0,0,1,316.0,1.0


In [21]:
df.isna().sum()

creation_time                    0
creation_source                  0
last_session_creation_time    3177
opted_in_to_mailing_list         0
enabled_for_marketing_drip       0
org_id                           0
invited_by_user_id            5583
adopted_user                     0
dtype: int64

In [19]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 12000 entries, 0 to 11999
Data columns (total 8 columns):
 #   Column                      Non-Null Count  Dtype  
---  ------                      --------------  -----  
 0   creation_time               12000 non-null  object 
 1   creation_source             12000 non-null  object 
 2   last_session_creation_time  8823 non-null   float64
 3   opted_in_to_mailing_list    12000 non-null  int64  
 4   enabled_for_marketing_drip  12000 non-null  int64  
 5   org_id                      12000 non-null  int64  
 6   invited_by_user_id          6417 non-null   float64
 7   adopted_user                12000 non-null  float64
dtypes: float64(3), int64(3), object(2)
memory usage: 750.1+ KB


In [20]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
last_session_creation_time,8823.0,1.379279e+09,1.953116e+07,1.338452e+09,1.363195e+09,1.382888e+09,1.398443e+09,1.402067e+09
opted_in_to_mailing_list,12000.0,2.495000e-01,4.327417e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
enabled_for_marketing_drip,12000.0,1.493333e-01,3.564316e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00
org_id,12000.0,1.418846e+02,1.240567e+02,0.000000e+00,2.900000e+01,1.080000e+02,2.382500e+02,4.160000e+02
invited_by_user_id,6417.0,5.962957e+03,3.383762e+03,3.000000e+00,3.058000e+03,5.954000e+03,8.817000e+03,1.199900e+04
adopted_user,12000.0,1.380000e-01,3.449144e-01,0.000000e+00,0.000000e+00,0.000000e+00,0.000000e+00,1.000000e+00


In [22]:
df.dtypes

creation_time                  object
creation_source                object
last_session_creation_time    float64
opted_in_to_mailing_list        int64
enabled_for_marketing_drip      int64
org_id                          int64
invited_by_user_id            float64
adopted_user                  float64
dtype: object

In [24]:
df.creation_source.value_counts()

ORG_INVITE            4254
GUEST_INVITE          2163
PERSONAL_PROJECTS     2111
SIGNUP                2087
SIGNUP_GOOGLE_AUTH    1385
Name: creation_source, dtype: int64

In [30]:
df.invited_by_user_id.value_counts()

10741.0    13
2527.0     12
1525.0     11
11770.0    11
2308.0     11
           ..
1390.0      1
5445.0      1
8526.0      1
4695.0      1
5450.0      1
Name: invited_by_user_id, Length: 2564, dtype: int64